In [1]:
%matplotlib notebook
%run preprocess.ipynb
%run models.ipynb

In [2]:
import os
import numpy as np
import shutil
import time
import json
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision as tv
import nntools as nt
import torch

In [105]:
def train_and_validation(model, dataloader, lr=0.001, rho=0.9, weight_decay, mode='training'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    running_loss, running_accuracy, num_updates = 0.0, 0.0, 0.0
    model.train() 
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.9)
    
    # Iterate over data.
    for image, question, answer in dataloader:
        if (device == 'cuda'):
            image, question, answer = image.cuda(), question.cuda(), answer.cuda()

        image, question, answer = Variable(image), Variable(question), Variable(answer)

        # zero grad
        optimizer.zero_grad()
        predicted_answer = model(image, question)

        _, class_indices = torch.max(answer, 1)        

        answer = torch.tensor(answer, dtype=torch.long, device=device)
        
        loss = criterion(predicted_answer, class_indices)
        
        if(mode=='training'):
            # backward + optimize
            loss.backward()
            nn.utils.clip_grad_norm(model.parameters(), 2, 10.0)
            optimizer.step()

        # statistics
        with torch.no_grad():
            running_loss += loss.item()
            running_accuracy += torch.sum((y_pred == class_indices).data)
    
        num_updates += 1

    loss = running_loss / num_updates
    acc = (running_accuracy / len(dataloader.dataset)) * 100
    print('Train Loss: {:.4f} Acc: {:2.3f} ({}/{})'.format(loss, acc, running_accuracy, num_updates))
    
    return loss, acc

In [106]:
def train_model(model, train_loader, validation_loader, save_dir, num_epochs=25, best_accuracy=0, start_epoch=0):
    best_model_wts = model.state_dict()
    best_acc = best_accuracy
    
    training_loss_history = []
    training_accuracy_history = []

    validation_loss_history = []
    validation_accuracy_history = []

    for epoch in range(start_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        train_loss, train_acc = train_and_validation(model, train_loader, mode='training') 
        training_loss_history.append(train_loss)
        training_accuracy_history.append(train_acc)    
        print('Epoch= ' + str(epoch) + ' Train Loss= ' + str(train_loss))

#         validation_begin = time.time()
#         val_loss, val_acc = train_and_validation(model, train_loader, mode='validation')
#         validation_loss_history.append(val_loss)
#         validation_accuracy_history.append(val_acc)  
#         print('Epoch= ' + str(epoch) + ' Validation Loss= ' + str(val_loss))

        # deep copy the model
#         is_best = val_acc > best_acc
#         if is_best:
#             best_acc = val_acc
#             best_model_wts = model.state_dict()

#         save_checkpoint(save_dir, {
#             'epoch': epoch,
#             'best_acc': best_acc,
#             'state_dict': model.state_dict(),
#             'optimizer': optimizer.state_dict(),
#             'training_history': [training_loss_history, training_accuracy_history],
#             'validation_history': [validation_loss_history, validation_accuracy_history]
#         }, is_best)

    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)

    return model

In [107]:
def save_checkpoint(save_dir, state, is_best):
    savepath = save_dir + '/' + 'checkpoint.pth.tar'
    torch.save(state, savepath)
    if is_best:
        shutil.copyfile(savepath, save_dir + '/' + 'model_best.pth.tar')

In [108]:
indices = np.random.permutation(len(train))
train_ind = indices[:int(len(train)*0.8)]
val_ind = indices[int(len(train)*0.8):]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_ind)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_ind)

In [109]:
train_loader = torch.utils.data.DataLoader(train, batch_size=10, pin_memory=True, sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train, batch_size=10, pin_memory=True, sampler=val_sampler)

In [110]:
san = SAN(num_classes=1000, batch_size=10, vocab_size=len(train.vocab_q), embedding_dim=1000,
          output_vgg=1024, input_attention=1024, output_attention=512)
san = san.to('cuda')

In [111]:
train_model(san, train_loader, val_loader, 'test', num_epochs=1, best_accuracy=0, start_epoch=0)

Epoch 0/0
----------


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


KeyboardInterrupt: 

In [ ]:
a = torch.Tensor([0, 0, 0, 1]).cuda()
torch.max(a, 1)